In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/returnrate/sample_submission.csv
/kaggle/input/returnrate/test.csv
/kaggle/input/returnrate/train.csv


In [2]:
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
def prepareData( train_df, test_df):
    print(train_df.info())
    data = train_df.values
    test_data = test_df.values
    
    #if not train:
    #    data = np.concatenate( (data , np.zeros(data.shape[0])) , axis=1)
    X = data[: , 1:data.shape[1]-1]
    Y = data[: , data.shape[1]-1]
    print(X.shape, Y.shape)
    X_Test = test_data[: , 1:test_data.shape[1]-1]
    
    
    labEncoder = LabelEncoder()
    labEncoder.fit(Y)
    label_y = Y 
    
    final_X = None
    Test_X = None
    for cnt in range(0 , X.shape[1]):
        print(train_df.columns[cnt+1])
        if train_df.columns[cnt+1].__contains__("status") or train_df.columns[cnt+1].__contains__("hedge_value") or train_df.columns[cnt+1].__contains__("indicator_code") or train_df.columns[cnt+1].__contains__("bought") or train_df.columns[cnt+1].__contains__("libor_rate") or train_df.columns[cnt+1].__contains__("sold") or train_df.columns[cnt+1].__contains__("start_date") or train_df.columns[cnt+1].__contains__("euribor_rate") or train_df.columns[cnt+1].__contains__("creation_date") or train_df.columns[cnt+1].__contains__("sell_date"):
            print("non obj")
            
            curFeature = X[:,cnt]
            curFeature = curFeature.reshape(X.shape[0], 1)
            
            curTestFeature = X_Test[:,cnt]
            curTestFeature = curTestFeature.reshape(X_Test.shape[0], 1)
        else:
            print("obj")
            labEncoder = LabelEncoder()
            labEncoder.fit(np.append(X[:,cnt] , X_Test[:,cnt]))
            curFeature = labEncoder.transform(X[:,cnt])
            curFeature = curFeature.reshape(X.shape[0], 1)
            
            curTestFeature = labEncoder.transform(X_Test[:,cnt])
            curTestFeature = curTestFeature.reshape(X_Test.shape[0], 1)
            
            
            

        if final_X is None:
            final_X = curFeature
        else:
            final_X = np.concatenate((final_X , curFeature) , axis=1)
        
        
        if Test_X is None:
            Test_X = curTestFeature
        else:
            Test_X = np.concatenate((Test_X , curTestFeature) , axis=1)
        
    seed = 8
    test_size = .1
    X_train, X_test, Y_train, Y_test = train_test_split(final_X, label_y, test_size=test_size,
                                                        random_state=seed)

    

    return X_train, X_test, Y_train, Y_test , Test_X

In [4]:
def prepareDataByPandas(train_df , test_df):
    modArr = dict()
    for colNam in train_df.columns:
        mod = train_df[colNam].mode()[0]
        modArr[colNam] = mod
    
    for colNam in train_df.columns:
        train_df[colNam].fillna(modArr[colNam] , inplace=True)
    
    for colNam in test_df.columns:
        test_df[colNam].fillna(modArr[colNam] , inplace=True)
    
    
    
    return prepareData(train_df,test_df)

In [5]:
train = pd.read_csv('../input/returnrate/train.csv')
test = pd.read_csv('../input/returnrate/test.csv')
sub = pd.read_csv('../input/returnrate/sample_submission.csv')

train.rename(columns={"return": "returnrate"}, inplace=True)
test['returnrate'] = 0

train.drop("portfolio_id", axis = 1, inplace = True)
test.drop("portfolio_id", axis = 1, inplace = True)

train.drop("desk_id", axis = 1, inplace = True)
test.drop("desk_id", axis = 1, inplace = True)

train.drop("office_id", axis = 1, inplace = True)
test.drop("office_id", axis = 1, inplace = True)

print(train.head())
print(test.head())

  pf_category  start_date         sold country_code  euribor_rate currency  \
0           B    20040720  110000000.0            T       0.02074      USD   
1           A    20040709  176671000.0            N       0.02074      GBP   
2           A    20040723   56474000.0            T       0.02074      USD   
3           A    20040609  164813000.0            T       0.02074      USD   
4           B    20040609  140800000.0            T       0.02074      USD   

   libor_rate        bought  creation_date indicator_code  sell_date type  \
0    2.332216  1.098097e+08       20040720            NaN   20040812    B   
1    5.269617  1.760084e+08       20040723            NaN   20040812    C   
2    2.332216  5.637953e+07       20040723            NaN   20040817    A   
3    2.332216  1.645088e+08       20040723            NaN   20040713    A   
4    2.332216  1.405402e+08       20040723            NaN   20040713    B   

  hedge_value status  returnrate  
0         NaN    NaN     0.02496 

In [6]:
x_train, x_test, y_train, y_test , actual_test = prepareDataByPandas(train , test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9366 entries, 0 to 9365
Data columns (total 15 columns):
pf_category       9366 non-null object
start_date        9366 non-null int64
sold              9366 non-null float64
country_code      9366 non-null object
euribor_rate      9366 non-null float64
currency          9366 non-null object
libor_rate        9366 non-null float64
bought            9366 non-null float64
creation_date     9366 non-null int64
indicator_code    9366 non-null bool
sell_date         9366 non-null int64
type              9366 non-null object
hedge_value       9366 non-null bool
status            9366 non-null bool
returnrate        9366 non-null float64
dtypes: bool(3), float64(5), int64(3), object(4)
memory usage: 905.6+ KB
None
(9366, 13) (9366,)
start_date
non obj
sold
non obj
country_code
obj
euribor_rate
non obj
currency
obj
libor_rate
non obj
bought
non obj
creation_date
non obj
indicator_code
non obj
sell_date
non obj
type
obj
hedge_value
non obj
status

In [7]:
#model = LGBMRegressor(max_depth=3,silent=False)
model = LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.01, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, max_depth=3)
model.fit(x_train, y_train)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.2319, feature_fraction_seed=9,
              importance_type='split', learning_rate=0.01, max_bin=55,
              max_depth=3, min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=6, min_split_gain=0.0,
              min_sum_hessian_in_leaf=11, n_estimators=720, n_jobs=-1,
              num_leaves=5, objective='regression', random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [8]:
y_pred = model.predict(x_test)

In [9]:
np.concatenate((y_pred.reshape(len(y_pred) ,1) , y_test.reshape(len(y_pred) ,1)), axis=1)

def Average(lst): 
    return sum(lst) / len(lst) 

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

# Mean of y_test
print("Mean: %.2f"
      % Average(y_test))

print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.00
Mean: 0.01
Variance score: 0.96
